In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, GRU
from keras.layers import Dense, Embedding, Bidirectional, Dropout, Flatten
from keras.optimizers import Adam, SGD
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

2024-06-11 08:28:36.527186: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [7]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
print('Training Dataset contain {} samples'.format(train.shape[0]))
print('Testing Dataset contain {} samples'.format(test.shape[0]))

Training Dataset contain 7613 samples
Testing Dataset contain 3263 samples


We will drop 'id', 'keyword' and 'location columns as we are going to train an RNN only on text data.

In [12]:
train = train.drop(['id', 'keyword', 'location'], axis=1)
test = test.drop(['id', 'keyword', 'location'], axis=1)

In [14]:
y_train =  train['target'].values
X_train = train.drop(['target'], axis=1).values.reshape(len(train),)
X_test = test['text'].values.reshape(len(test),)

In [16]:
X_train

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

**Preprocessing**

In [19]:
total_tweets = np.concatenate((X_train, X_test))
print('Total tweets : ', len(total_tweets))

Total tweets :  10876


In [21]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(total_tweets)

# Vocbvulary Size
vocab_size = len(tokenizer.word_index) + 1
print('Size of Vocabulary : ', vocab_size)

Size of Vocabulary :  29624


In [23]:
# Maximum length for padding sequence
maxlen = max(len(x.split()) for x in total_tweets)
print('Maximum length of tweet : ', maxlen)

Maximum length of tweet :  31


In [25]:
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

print('Text before tokenized')
print(X_train[0])
print('\nText after tokenized')
print(X_train_token[0])

Text before tokenized
Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all

Text after tokenized
[115, 5954, 25, 4, 872, 8, 22, 253, 152, 1818, 3839, 89, 42]


In [27]:
X_train_pad = pad_sequences(X_train_token, maxlen=maxlen, padding='post')
X_test_pad = pad_sequences(X_test_token, maxlen=maxlen, padding='post')

print('Tokenized text before padding')
print(X_train_token[0])
print('\nTokenized text after padding')
print(X_train_pad[0])

Tokenized text before padding
[115, 5954, 25, 4, 872, 8, 22, 253, 152, 1818, 3839, 89, 42]

Tokenized text after padding
[ 115 5954   25    4  872    8   22  253  152 1818 3839   89   42    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]


In [29]:
hidden_units = 128
embed_units = 100

model = Sequential()
model.add(Embedding(vocab_size, embed_units, input_length = maxlen))
model.add(Bidirectional(LSTM(hidden_units)))
model.add(Dropout(0.2))
#model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

2024-06-11 08:39:05.084447: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 31, 100)           2962400   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              234496    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                        

2024-06-11 08:39:05.359235: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-11 08:39:05.360734: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-11 08:39:05.361618: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [31]:
learning_rate = 0.0001

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [33]:
batch_size = 512
num_itr = 5

model_history = model.fit(X_train_pad, y_train, 
                          batch_size=batch_size, 
                          epochs=num_itr, 
                          validation_split=0.2)

Epoch 1/5


2024-06-11 08:39:33.460926: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-11 08:39:33.462989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-11 08:39:33.464495: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

12/12 [==============================] - ETA: 0s - loss: 0.6718 - accuracy: 0.5760

2024-06-11 08:39:41.758159: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-11 08:39:41.759691: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-11 08:39:41.761143: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

12/12 [==============================] - 9s 526ms/step - loss: 0.6718 - accuracy: 0.5760 - val_loss: 0.6563 - val_accuracy: 0.5496
Epoch 2/5
12/12 [==============================] - 6s 537ms/step - loss: 0.5663 - accuracy: 0.7458 - val_loss: 0.5318 - val_accuracy: 0.7584
Epoch 3/5
12/12 [==============================] - 6s 533ms/step - loss: 0.3462 - accuracy: 0.8616 - val_loss: 0.4901 - val_accuracy: 0.7866
Epoch 4/5
12/12 [==============================] - 6s 507ms/step - loss: 0.2045 - accuracy: 0.9209 - val_loss: 0.4994 - val_accuracy: 0.7722
Epoch 5/5
12/12 [==============================] - 6s 544ms/step - loss: 0.1093 - accuracy: 0.9627 - val_loss: 0.7586 - val_accuracy: 0.7170


In [34]:
pred = model.predict(X_test_pad)

2024-06-11 08:40:17.067262: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-11 08:40:17.068875: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-11 08:40:17.070701: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

102/102 [==============================] - 3s 25ms/step


In [37]:
sub=pd.read_csv("sample_submission.csv")
sub["target"] = pred
sub["target"]=sub["target"].apply(lambda x : 0 if x<=.5 else 1)

In [39]:
sub.to_csv("submit_1.csv", index=False)